In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
TRAIN_DIR = "data/prepared-data/<folder_name>/train"
VAL_DIR   = "data/prepared-data/<folder_name>/val"

IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
EPOCHS = 10

In [ ]:
# eğitim sırasında augment, normalize ve grayscale uygulama
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=15
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

In [ ]:
# Eğer grayscale yapmak istersen, flow_from_directory ile color_mode='grayscale' kullanabilirsin
# Ama MobileNetV3 3 kanal RGB bekliyor, bu yüzden grayscale → RGB yapıyoruz
def preprocess_grayscale_rgb(img):
    img = tf.image.rgb_to_grayscale(img)  # grayscale
    img = tf.image.grayscale_to_rgb(img)  # tekrar 3 kanal
    return img

In [ ]:
# Custom preprocessing ile ImageDataGenerator kullanmak için:
def custom_preprocessing(img):
    img = tf.cast(img, tf.float32)
    img = preprocess_grayscale_rgb(img)
    img = img / 255.0  # normalize
    return img

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

In [ ]:
base_model = tf.keras.applications.MobileNetV3Large(
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = True

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS
)

In [ ]:
model.save("mobilenetv3_aug_gray_norm.h5")